In [1]:
import os 

os.chdir("/home/theovincent/MVA/3DPointCloud/3DPointCloudClassification")

## Combine the point cloud and the dataset

In [2]:
import numpy as np

from classifier_3D.utils.ply_file import write_ply
from range_net import KITTI_TO_CITY_NUMBERS

city_points = np.fromfile("/home/theovincent/MVA/3DPointCloud/RangeNet++/lidar-bonnetal/train/tasks/semantic/data_city/sequences/00/velodyne/000.bin", dtype=np.float32).reshape((-1, 4))
kitty_labels = np.fromfile("/home/theovincent/MVA/3DPointCloud/RangeNet++/lidar-bonnetal/train/tasks/semantic/data_city/sequences/00/labels/000.label", dtype=np.int32)
city_labels = np.array([KITTI_TO_CITY_NUMBERS[label] for label in kitty_labels]).astype(np.int32)

write_ply("data/MiniLille1_combined_sample_raw.ply", (city_points, city_labels), ["x", "y", "z", "verticality", "class"])

True

In [30]:
import numpy as np

from classifier_3D.utils.ply_file import read_ply, write_ply
from classifier_3D.utils.path import get_data_path

from range_net import PATH_INDEXES_TO_KEEP, KITTI_TO_CITY_NUMBERS, CITY_NUMBERS_TO_LABELS, MAX_DISTANCE

is_train_data = True
file = "MiniLille1"
folder = "00"
path_to_predictions = f"/home/theovincent/MVA/3DPointCloud/RangeNet++/lidar-bonnetal/train/tasks/semantic/preds/sequences/{folder}/predictions"

point_cloud_path = get_data_path(
    f"{file}_with_features", is_train_data
)
point_cloud, _ = read_ply(point_cloud_path)
points = np.vstack((point_cloud["x"], point_cloud["y"], point_cloud["z"])).T.astype(
        np.float32
    )

list_path_predictions = sorted([f"{path_to_predictions}/{file_name}" for file_name in os.listdir(path_to_predictions) if file_name[-6:] == ".label"])

weighted_targets = np.zeros((points.shape[0], len(CITY_NUMBERS_TO_LABELS)), dtype=np.float32)

for idx_prediction, path_prediction in enumerate(list_path_predictions):
    path_to_indexes = f"{PATH_INDEXES_TO_KEEP}/{folder}/{os.path.split(path_prediction)[-1].replace('.label', '.npy')}"
    path_to_sample = path_prediction.replace("preds", "data_city").replace("predictions", "velodyne").replace('.label', '.bin')

    indexes = np.load(path_to_indexes).astype(bool)
    sample = np.fromfile(path_to_sample, dtype=np.float32).reshape((-1, 4))[:, :3]
    ranges = np.linalg.norm(sample, axis=1).astype(np.float32)
    kitti_prediciton = np.fromfile(path_prediction, dtype=np.int32)
    city_prediciton = np.array([KITTI_TO_CITY_NUMBERS[label] for label in kitti_prediciton]).astype(np.int32)

    weights = np.exp(- 10 * ranges / MAX_DISTANCE)

    weighted_targets[indexes, city_prediciton] +=  weights

In [35]:
write_ply("data/MiniLille1_combined_sample_raw.ply", (points, weighted_targets.argmax(axis=1).astype(np.int32)), ["x", "y", "z", "class"])

True

To do:
- merge predictions
- test on train set
- do test set
- create colab to infer 